In [ ]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from eos.data_io.config import drivers, trucks_by_id, drivers_by_id


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

ts = pd.to_datetime(datetime.now())
ts_ind = ts + pd.to_timedelta(np.arange(2), 'H')
ts_ind
a = np.array([[1, 2, 3], [4, 5, 6]])
df = pd.DataFrame(a, index=ts_ind, columns=['c1', 'c2', 'c3'])
df
df.index[0]


In [ ]:
ts
ts_end = pd.to_datetime(datetime.now())
ts_end
span = ts_end - ts
span
span_each_row = span / 4
span_each_row
np.linspace(0, 4, 5)
ts_ser = ts + pd.to_timedelta(np.linspace(1, 4, 4) * span_each_row, unit='s')
ts_ser


In [ ]:
df_ts = pd.DataFrame(np.array([ts]), columns=['timestamp'])
df_ts

In [ ]:
s = np.arange(10)
s

### Generate state

In [ ]:
ss = np.arange(12)
a1 = ss[:4].tolist()
a2 = ss[4:8].tolist()
a3 = ss[8:].tolist()
ss = [a1, a2, a3]
ss
df_s = pd.DataFrame(ss, columns=['t', 'v', 'p', 'b'])  # .set_index('t')
df_s

In [ ]:
np.arange(0, 4 * 20, 20)

In [ ]:
ts_ind = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ts_ind
ss = np.arange(12)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
df_ss = pd.DataFrame(
    {'timestep': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3}
)  # .set_index('timestep')
df_ss.columns.name = 'qtuple'
df_ss
df_ss[['velocity', 'thrust', 'brake']]

In [ ]:
npa_ss_flatten = df_ss[['velocity', 'thrust', 'brake']].to_numpy().flatten()
npa_ss_flatten

In [ ]:
ui_t = df_ss.loc[:, ['velocity', 'thrust']]
ui_t
power_t = ui_t.prod(axis=1)
power_t
power_t.sum()

In [ ]:
state = df_ss.stack().swaplevel(0, 1)
state.name = 'state'
state.index.names = ['rows', 'idx']
state.sort_index(inplace=True)
state

In [ ]:
type(state)
state

### generate action

In [ ]:
from functools import reduce

a = len(ss) + np.arange(15)
speed_ser = pd.Series(np.linspace(40, 60, 3), name='speed')
# row_ser
row_dict = {f'r{i}': a[i * 5 : i * 5 + 5] for i in np.arange(3)}
row_dict
row_array = a.reshape(3, 5).transpose()
row_array
rows_df = pd.DataFrame(row_array)
rows_df.columns = [f'r{i}' for i in np.arange(3)]
rows_df


In [ ]:

ts_ind = ts + pd.to_timedelta(np.arange(5 * 20, 8 * 20, 20), 'ms')
ts_ser = pd.Series(ts_ind, name='timestep')
throttle_ser = pd.Series(np.linspace(0, 1.0, 5), name='throttle')
# throttle_ser
dfs = [rows_df, ts_ser, speed_ser, throttle_ser]
action = (
    reduce(
        lambda left, right: pd.merge(
            left, right, how='outer', left_index=True, right_index=True
        ),
        dfs,
    )
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)

action.name = 'action'
action.index.names = ['rows', 'idx']
action.index
action

In [ ]:
reward = (
    pd.DataFrame({'work': len(ss) + len(a), 'timestep': ts_ind[0]}, index=[0])
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
# reward_index = (reward.name,  ts_ind[0], 0)
reward.index.names = ['rows', 'idx']
reward.name = 'reward'
reward.index
reward
# reward.name = 'reward'

In [ ]:
reward['work'][0]
reward.loc[('work', 0)]

In [ ]:
ts_ind = ts + pd.to_timedelta(5, 's') + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ts_ind
ss = (
    np.arange(12) + len(ss) + len(a) + len(reward) - 1
)  # exclude the timestamp in reward
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
nstate = (
    pd.DataFrame({'timestep': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3})
    # .set_index('timestamp')
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
nstate.name = 'nstate'
nstate.index.names = ['rows', 'idx']
len(nstate)
nstate

In [ ]:
timestamp = pd.Series([ts], name='timestamp')
timestamp.index = pd.MultiIndex.from_product(
    [timestamp.index, [0]], names=['rows', 'idx']
)
timestamp.index
state.index
action.index
reward.index
nstate.index

In [ ]:
timestamp_index = (timestamp.name, '', 0)
# timestamp_index
state_index = [(state.name, *i) for i in state.index]
# state_index
reward_index = [(reward.name, *i) for i in reward.index]
# reward_index
action_index = [(action.name, *i) for i in action.index]
# action_index
nstate_index = [(nstate.name, *i) for i in nstate.index]
# nstate_index

multiindex = pd.MultiIndex.from_tuples(
    [timestamp_index, *state_index, *action_index, *reward_index, *nstate_index]
)
multiindex


In [ ]:
observation_list = [timestamp, state, action, reward, nstate]
observation = pd.concat(observation_list)
observation.index = multiindex
observation
observation.dtype

In [ ]:
observation

In [ ]:

observation0 = observation.copy()
observation0.loc['timestamp', '', 0] = ts + pd.Timedelta(1, 'h')
observation1 = observation.copy()
observation1.loc['timestamp', '', 0] = ts + pd.Timedelta(2, 'h')
observation2 = observation.copy()
observation2.loc['timestamp', '', 0] = ts + pd.Timedelta(3, 'h')
observation3 = observation.copy()
observation3.loc['timestamp', '', 0] = ts + pd.Timedelta(4, 'h')
observation4 = observation.copy()
observation4.loc['timestamp', '', 0] = ts + pd.Timedelta(5, 'h')
observation_list = [
    observation0,
    observation1,
    observation2,
    observation3,
    observation4,
]


### extract state action reward nstate from list of observations

In [ ]:
idx = pd.IndexSlice
observation_list[0]
observation_list[0].index
observation_list[0].loc[idx['state', ['velocity', 'thrust', 'brake']]].values
observation_list[0].loc[idx['action', ['r0', 'r1', 'r2']]].values
observation_list[0].loc[idx['reward', ['work']]].values
observation_list[0].loc[idx['nstate', ['velocity', 'thrust', 'brake']]].values

In [ ]:
idx = pd.IndexSlice
state = []
action = []
reward = []
nstate = []
for observation in observation_list:
    state.append(observation.loc[idx['state', ['velocity', 'thrust', 'brake']]].values)
    action.append(observation.loc[idx['action', ['r0', 'r1', 'r2']]].values)
    reward.append(observation.loc[idx['reward', ['work']]].values)
    nstate.append(
        observation.loc[idx['nstate', ['velocity', 'thrust', 'brake']]].values
    )
npa_state = np.stack(state)
npa_action = np.stack(action)
npa_reward = np.stack(reward)
npa_nstate = np.stack(nstate)
npa_state
npa_action
npa_reward
npa_nstate

In [ ]:

dfs_epi = pd.concat(observation_list, axis=1).transpose()
dfs_epi.columns.names = ['qtuple', 'rows', 'idx']
dfs_epi.columns
dfs_epi


In [ ]:
dfs_episode1 = dfs_epi.copy().sort_index(axis=1)
dfs_episode1.set_index(('timestamp', '', 0), inplace=True)
dfs_episode1.index.name = 'timestamp'
idx = pd.IndexSlice
dfs_episode1

In [ ]:
dfs_episode1.loc[:, idx['action':'state', :, :, :]] = dfs_episode1.loc[
    :, idx['action':'state', :, :, :]
].astype('int')
dfs_episode1
dfs_episode1.dtypes
dfs_episode1.columns

### Convert columns types to float

In [ ]:
dfs_epi
dfs_episode = dfs_epi.copy()
dfs_episode.index
dfs_episode.set_index(('timestamp', '', 0), inplace=True)
dfs_episode.sort_index(axis=1, inplace=True)
dfs_episode.index
dfs_episode


In [ ]:
dfs_episode.index.name = 'timestamp'
idx = pd.IndexSlice


state_cols_float = [('state', col) for col in ['brake', 'thrust', 'velocity']]
action_cols_float = [('action', col) for col in ['r0', 'r1', 'r2', 'speed', 'throttle']]
reward_cols_float = [('reward', 'work')]
nstate_cols_float = [('nstate', col) for col in ['brake', 'thrust', 'velocity']]
for col in action_cols_float + state_cols_float + reward_cols_float + nstate_cols_float:
    dfs_episode[col[0], col[1]] = dfs_episode[col[0], col[1]].astype(
        'float'
    )  # float16 not allowed in parquet
dfs_episode
dfs_episode.dtypes
# dfs_episode.columns
# dfs_epi
# dfs_episode


## prepend two levels of index "vehicle" and "driver" to the DataFrame object

In [ ]:
episodestart = ts - pd.Timedelta(1, 'h')
dfs_epi = pd.concat([dfs_episode], keys=[ts], names=['episodestart'])

dfs_epi = pd.concat(
    [dfs_epi], keys=[drivers_by_id['wang-cheng'].pid], names=['driver']
)
dfs_epi = pd.concat(
    [dfs_epi], keys=[trucks_by_id['VB7'].vid], names=['vehicle']
)

In [ ]:
dfs_epi.index
dfs_epi.columns
dfs_epi
dfs_epi.dtypes


## Add a level of index for episode

In [ ]:

# dfs_episode = dfs_episode.swaplevel(1, 0, axis=0)
# dfs_episode = dfs_episode.swaplevel(1, 2, axis=0)
dfs_epi.sort_index(inplace=True)
dfs_epi.index
dfs_epi.columns
dfs_epi

In [ ]:
dfs_epi.dtypes

In [ ]:
vb7 = trucks_by_id['VB7']

In [ ]:
from eos.data_io.struct import PoolQuery

query = PoolQuery(
    site=trucks_by_id['VB7'].site,
    vehicle=trucks_by_id['VB7'].vid,
    driver=drivers_by_id['zheng-longfei'].pid,
    start=episodestart,
    end=ts,
)
isinstance(query, dict)

In [ ]:
ts_new = pd.to_datetime((datetime.now()))
ts_new
episodestart = ts_new - pd.Timedelta(2, 'd')
episodestart

dfs_episode0 = dfs_epi.copy()
dfs_episode0.index = dfs_episode0.index.set_levels([episodestart], level='episodestart')
dfs_episode0.index = dfs_episode0.index.set_levels(
    [[trucks_by_id['VB7'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode1 = dfs_epi.copy()
dfs_episode1.index = dfs_episode1.index.set_levels(
    [episodestart - pd.Timedelta(3, 'd')], level='episodestart'
)
dfs_episode1.index = dfs_episode1.index.set_levels(
    [[trucks_by_id['MP73'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode2 = dfs_epi.copy()
dfs_episode2.index = dfs_episode2.index.set_levels(
    [episodestart - pd.Timedelta(4, 'd')], level='episodestart'
)
dfs_episode2.index = dfs_episode2.index.set_levels(
    [[trucks_by_id['VB7'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode3 = dfs_epi.copy()
dfs_episode3.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(5, 'd')], level='episodestart'
)
dfs_episode3.index = dfs_episode3.index.set_levels(
    [[trucks_by_id['MP73'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode0



In [ ]:
from functools import reduce

episodes = [dfs_epi, dfs_episode0, dfs_episode1, dfs_episode2, dfs_episode3]
try:
    dfs_episode_all = reduce(
        lambda left, right,: pd.concat([left, right], axis=0), episodes
    )
except Exception as e:
    print(e)
# dfs_episode_all.sort_index(inplace=True)
# dfs_episode_all = dfs_episode_all.swaplevel(1, 0, axis=0)
# dfs_episode_all = dfs_episode_all.swaplevel(1, 2, axis=0)
dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all[['state', 'action', 'reward', 'nstate']]
dfs_episode_all
dfs_episode_all.index
dfs_episode_all.columns

In [ ]:
dfs_episode_all

## Make MultiIndex to Rows by adding levels of "Vehicle" and "Driver"

###  reset index to move vehicle and driver to columns, preprocessing for dask manipulation

In [ ]:
dfs_episodes = dfs_episode_all.reset_index(
    level=['vehicle', 'driver']
)  # unstack vehicle and driver to columns in level 0 with default '' in level 1
dfs_episodes
# dfs_episodes.columns
# dfs_episodes.dtypes
# dfs_episode_all

In [ ]:
dfs_episodes.dtypes

In [ ]:
dfs_episodes.columns

In [ ]:

dfs_episodes['vehicle'] = dfs_episodes['vehicle'].astype("category")
dfs_episodes['driver'] = dfs_episodes['driver'].astype("category")

dfs_episodes_sorted = dfs_episodes.sort_index(
    axis=1, level=[0, 1, 2], ascending=[True, True, True]
)
dfs_episodes_sorted.sort_index(axis=0, inplace=True)
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=1, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=0, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted.sort_index(axis=1,inplace=True)
dfs_episodes_sorted
dfs_episodes_sorted.columns

# dfs_episodes[['driver'], ['vehicle']] = dfs_episodes[['driver', 'vehicle']].apply(lambda x: x.cat.codes)

In [ ]:
dfs_episodes_sorted
# dfs_episode.loc[:,'state']
dfs_slice = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['vehicle'] == 'MP73', ['vehicle', 'driver', 'state', 'reward']
]
dfs_slice

# Saved episodes as parquet in multi-level folders "vehicle/driver/episode/tuple.parquet"

In [ ]:
import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
from datetime import datetime

import pyarrow as pa
import pyarrow.parquet as pq
import platform

print('Python: ', platform.python_version())
print('pandas: ', pd.__version__)
print('pyarrow: ', pa.__version__)


In [ ]:
os.chdir('../../data')

In [ ]:

os.getcwd()

## create parquet files

In [ ]:
dfs_episodes_sorted
dfs_episodes_sorted.index
dfs_episodes_sorted.columns
dfs_episodes_sorted.dtypes

In [ ]:
dfs_complete_episodes = dfs_episodes_sorted.reset_index('episodestart')
dfs_complete_episodes
dfs_complete_episodes.index
dfs_complete_episodes.columns

### encoding multiindex dataframe to flat index for Dask DataFrame

In [ ]:
dfs_complete_episodes.columns
cols = dfs_complete_episodes.columns.to_flat_index()
cols

## 3 level index
# cols_str = [
#     f'{x[0]}_{x[1]}_{x[2]}'
#     if (x[1] != '')
#     else f'{x[0]}__{x[2]}'
#     if (x[2] != '')
#     else f'{x[0]}'
#     for x in cols
# ]

## 4 level index
cols_str = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] != '')
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] == '')
    else f'{x[0]}__{x[2]}'
    if (x[2] != '')
    else f'{x[0]}__'
    for x in cols
]
cols_str

In [ ]:
# encoding index and columns
dfs_complete_episodes_flat = dfs_complete_episodes.copy()
cols = dfs_complete_episodes.columns.to_flat_index()
dfs_complete_episodes_flat.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] != '')
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] == '')
    else f'{x[0]}__{x[2]}'
    if (x[2] != '')
    else f'{x[0]}__'
    for x in cols
]
dfs_complete_episodes_flat



In [ ]:

dfs_complete_episodes_flat
dfs_complete_episodes_flat.dtypes

In [ ]:
dfs_complete_episodes_flat['vehicle__'].iloc[0:2].values.dtype
dfs_complete_episodes_flat['vehicle__'].dtype
dfs_complete_episodes_flat['state_velocity_0'].dtype


In [ ]:
try:
    ddf_episodes = dd.from_pandas(dfs_complete_episodes_flat, npartitions=1)
except Exception as e:
    print(e)

try:
    ddf_episodes.index
    ddf_episodes.columns
except Exception as e:
    print(e)

ddf_episodes

In [ ]:

custom_metadata = {
    b'eos': b'{"length":[3, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}
custom_metadata

In [ ]:
try:
    with ProgressBar():
        ddf_episodes.to_parquet(
            'eos_new_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle__', 'driver__', 'episodestart__'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

### Read saved parquet file

In [ ]:
ddf = dd.read_parquet(
    'eos_new_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.head(5)
ddf.dtypes
ddf.npartitions
ddf.divisions
ddf._meta
ddf.compute()

### Get customized metadata with pyarrow

In [ ]:

import pyarrow as pa
import pyarrow.parquet as pq
import json

# print('pyarrow: ', pa.__version__)

# table = pq.read_table('ddf_episodes')
# table_meta = table.schema.metadata[b'eos']
# print(table_meta)
# # custom_metadata_info = json.loads(table.schema.metadata[b'eos'])
# # print(custom_metadata_info)
# try:
#     custom_meta_info = json.loads(table_meta)
# except Exception as e:
#     print(f'Exception: {e}')

# print(custom_meta_info)

# # Print formatted output of the dichtionary custom_meta_info:
# for key, val in custom_meta_info.items():
#     print('{:15}: {}'.format(key, val))

# ALTERNATE:

table_meta = pq.read_metadata('eos_complete_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = table_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

# table_meta = pq.read_metadata('ddf_episodes/_metadata')
# # print(table_meta)
# # print(table_meta.metadata[b'eos'])
# custom_meta_info = table_meta.metadata[b'eos']
# custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

### encoding flat_index of dask dataframe to multiindex

In [ ]:
df_episodes_read = ddf.compute()
df_episodes_read

In [ ]:
# decoding
multi_cols = [tuple(col.split('_')) for col in df_episodes_read.columns]
multi_cols
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_cols)
# multi_idx
multi_idx.names = ['qtuple', 'rows', 'idx']
driver_vehicle = [
    idx
    for idx in multi_idx
    if idx[0] == 'driver' or idx[0] == 'vehicle' or idx[0] == 'episodestart'
]
driver_vehicle
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_idx

In [ ]:
# pd.Series([1, np.nan,2,None])
# pd.Series([1,np.nan,2,None,pd.NA])

In [ ]:
# decoding columns
from datetime import datetime

multi_tpl = [tuple(col.split('_')) for col in df_episodes_read.columns]
# multi_tpl
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_idx
# multi_col
i1 = multi_col.get_level_values(0)
i1 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1
]  # convert index of level 2 type to int and '' if NA
i1
# i2 = multi_col.get_level_values(1).fillna('') # must be null string instead of the default pd.NA or np.nan
# i2 = [idx if isinstance(idx, int) else '' for idx in i2]
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i2
# i2 = [idx if isinstance(idx, str) else '' for idx in i2]
# i2.astype('int')
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA
i3


multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['qtuple', 'rows', 'idx']
multi_col


# i2[-1]
# i2.dropna()
# i2
# l2 = ['' if i==nan else i  for i in i2]
# l2
# i2= i2[:-2].astype('int')
# i2 = i2.append(pd.Index(['','']))
# l2 = [''] + list(l2.values[:-1])
# print(l2)
# arrays = [ for i2 in l2 for i1 in l1]
# l2 = l2[:-2].astype('int') + ['', '']
# multi_col.set_levels(ll, level=['qtuple','step'])
# driver_vehicle = [idx for idx in multi_idx if idx[0]=='driver' or idx[0]=='vehicle']
# multi_tpl =  [(col[0],)  if col[0]=='driver' or col[0]=='vehicle' else (col[0],int(col[1])) for col in multi_col]
# multi_idx1 = pd.MultiIndex.from_tuples(multi_idx1)
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_col

In [ ]:
# # decoding index
# multi_tuple = [tuple(idx.split('_')) for idx in df_episodes_read.index]
# multi_idx = pd.MultiIndex.from_tuples(multi_tuple)
# # multi_idx
# i0 = multi_idx.get_level_values(
#     0
# )  # must be null string instead of the default pd.NA or np.nan
# i0 = [
#     ''
#     if str(idx) in (str(pd.NA), 'nan', '')
#     else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
#     for idx in i0
# ]  # convert index of level 2 type to int and '' if NA
# i1 = multi_idx.get_level_values(
#     1
# )  # must be null string instead of the default pd.NA or np.nan
# i1 = [
#     ''
#     if str(idx) in (str(pd.NA), 'nan', '')
#     else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
#     for idx in i1
# ]  # convert index of level 2 type to int and '' if NA
# df_episodes_read_multiidx = df_episodes_read.copy()
# multi_idx = pd.MultiIndex.from_arrays([i0, i1], names=('episodestart', 'timestamp'))
#
# multi_idx.dtypes
# multi_idx
# len(multi_idx)

In [ ]:
df_episodes_read_multi = df_episodes_read.copy()
df_episodes_read_multi.columns = multi_col
# df_episodes_read_multi.index = multi_idx
df_episodes_read_multi
df_episodes_read_multi.columns
df_episodes_read_multi.index
df_episodes_read_multi.dtypes

### recover deep multiindex from columns ['vehicle','driver','episodestart']

In [ ]:
df_episodes_read_multi_setindex = df_episodes_read_multi.set_index(
    ['vehicle', 'driver', 'episodestart', df_episodes_read_multi.index]
)
# df_episodes_read_multi_setindex
# df_episodes_read_multi_setindex.sort_index(inplace=True)
# df_episodes_read_multi_setindex = df_episodes_read_multi_setindex.swaplevel(
#     1, 2, axis=0
# ).swaplevel(0, 1, axis=0)
# df_episodes_read_multi_setindex.index
# df_episodes_read_multi_setindex.columns
df_episodes_read_multi_setindex

## append new episode to the specific parquet file

In [ ]:
dfs_episode4 = dfs_epi.copy()
dfs_episode4.index = dfs_episode4.index.set_levels(
    [episodestart - pd.Timedelta(8, 'd')], level='episodestart'
)
dfs_episode4.index = dfs_episode4.index.set_levels(
    [[trucks_by_id['VB4'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
# dfs_episode4
# dfs_episode4 = dfs_episode4.swaplevel(1, 2, axis=0).swaplevel(0, 1, axis=0)
dfs_episode4

# dfs_episode5 = dfs_episode.copy()
# dfs_episode5.index = dfs_episode3.index.set_levels([episodestart-pd.Timedelta(9,'d')], level='episode')
# df_episodes_read_multi_setindex.columns
# df_episodes_read_multi_setindex

In [ ]:
dfs_episodes_new = pd.concat([df_episodes_read_multi_setindex, dfs_episode4], axis=0)
dfs_episodes_new.index.names = ['vehicle', 'driver','episodestart',  'timestamp']
dfs_episodes_new
dfs_episodes_new.columns
dfs_episodes_new.index

#### save new dataframe back to parquet

In [ ]:

os.getcwd()

In [ ]:
dfs_episodes_new_flat = dfs_episodes_new.copy()
dfs_episodes_new_flat = dfs_episodes_new_flat.reset_index(
    level=['vehicle', 'driver', 'episodestart']
)
dfs_episodes_new_flat.columns
dfs_episodes_new_flat.index
dfs_episodes_new_flat

In [ ]:
# encoding index
# from datetime import datetime

# flat_idx = dfs_episodes_new_flat.index.to_flat_index()
# flat_idx = [
#     f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
#     if x[1] != ''
#     else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
#     for x in flat_idx
# ]
# # flat_idx

# encoding columns
flat_cols = dfs_episodes_new_flat.columns.to_flat_index()
# flat_cols
# flat_cols = [
#     f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[3]}'
#     if (x[1] != '' and x[3] not in ('', pd.NaT, str(pd.NA)))
#     else f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
#     if (x[1] != '' and x[3] in ('', pd.NaT, str(pd.NA)))
#     else f'{x[0]}__{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
#     if (x[2] not in ('', pd.NaT, str(pd.NA)))
#     else f'{x[0]}___'
#     for x in flat_cols
# ]
flat_cols = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] not in ('', pd.NaT, str(pd.NA)))
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] in ('', pd.NaT, str(pd.NA)))
    else f'{x[0]}__{x[2]}'
    if (x[2] not in ('', pd.NaT, str(pd.NA)))
    else f'{x[0]}__'
    for x in flat_cols
]
flat_cols

In [ ]:

dfs_episodes_new_flat.columns = flat_cols
# dfs_episodes_new_flat.index = flat_idx
dfs_episodes_new_flat['vehicle__'] = dfs_episodes_new_flat['vehicle__'].astype(
    "category"
)
dfs_episodes_new_flat['driver__'] = dfs_episodes_new_flat['driver__'].astype("category")
dfs_episodes_new_flat['episodestart__'] = dfs_episodes_new_flat[
    'episodestart__'
].astype("datetime64[ns]")
# dfs_episodes_new_flat.index.name = 'episodestart_timestamp'
dfs_episodes_new_flat.index
dfs_episodes_new_flat.dtypes
# dfs_episodes_new_flat.columns.name = 'episodestart_timestamp'
dfs_episodes_new_flat.columns
dfs_episodes_new_flat


In [ ]:
try:
    ddf_episodes_new = dd.from_pandas(dfs_episodes_new_flat, npartitions=1)
except Exception as e:
    print(e)
ddf_episodes_new.dtypes
# dfs_episodes_new_flat


In [ ]:
os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[6, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_episodes_new.to_parquet(
            'eos_new_add_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle__', 'driver__', 'episodestart__'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

In [ ]:
ddf = dd.read_parquet(
    'eos_new_add_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('eos_episodes/_common_metadata')
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
for key, val in custom_meta_info.items():
    print(f'{key}: {val}')
ddf.dtypes
df = ddf.compute()

# decoding columns index
multi_tpl = [tuple(col.split('_')) for col in df.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i1 = ['' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1]
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA


multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['qtuple', 'rows', 'idx']
multi_col

# # decoding index
# multi_tuple = [tuple(idx.split('_')) for idx in df.index]
# multi_idx = pd.MultiIndex.from_tuples(multi_tuple)

# i0 = multi_idx.get_level_values(
#     0
# )  # must be null string instead of the default pd.NA or np.nan
# i0 = [
#     ''
#     if str(idx) in (str(pd.NA), 'nan', '')
#     else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
#     for idx in i0
# ]  # convert index of level 2 type to int and '' if NA
# i1 = multi_idx.get_level_values(
#     1
# )  # must be null string instead of the default pd.NA or np.nan
# i1 = [
#     ''
#     if str(idx) in (str(pd.NA), 'nan', '')
#     else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
#     for idx in i1
# ]  # convert index of level 2 type to int and '' if NA
# df_episodes_read_multiidx = df_episodes_read.copy()
# multi_idx = pd.MultiIndex.from_arrays([i0, i1], names=('episodestart', 'timestamp'))
#
# multi_idx.names = ['episodestart', 'timestamp']

df.columns = multi_col
# df.index = multi_idx

df = df.set_index(['vehicle', 'driver', 'episodestart',df.index])
df.index.dtypes
df.index
df
# len(multi_idx)

### add rows to a partition

In [ ]:

ddf = dd.read_parquet(
    'eos_new_add_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.index
ddf.npartitions
ddf.divisions
ddf.dtypes

# ddf['episodestart'] = ddf['episodestart'].astype('datetime64[ns]')  # error,  not allowed for dask dataframe
# ddf.assign(episodestart=ddf['episodestart'].astype('datetime64[ns]'))
ddf['episodestart__'] = ddf['episodestart__'].astype('datetime64[ns]')
ddf.dtypes
ddf.compute()

In [ ]:
dfs_episode4
dfs_episode4.index
dfs_episode4.columns

In [ ]:
episodestart

In [ ]:
dfs_episode5 = dfs_episode4.copy()
dfs_episode5.index = dfs_episode5.index.set_levels(
    [episodestart - pd.Timedelta(10, 'D')], level='episodestart'
)
dfs_episode5.index = dfs_episode5.index.set_levels(
    [[trucks_by_id['MP74'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
# dfs_episode5 = dfs_episode5.swaplevel(0,1, axis=0).swaplevel(1,2,axis=0)
dfs_episode5 = dfs_episode5.reset_index(level=['vehicle', 'driver', 'episodestart'])

dfs_episode5['vehicle'] = dfs_episode5['vehicle'].astype('category')
dfs_episode5['driver'] = dfs_episode5['driver'].astype('category')
# dfs_episode5['episodestart'] = dfs_episode5['episodestart'].astype('category')
dfs_episode5.dtypes


# encoding columns index
dfs_episode5.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] != '')
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] == '')
    else f'{x[0]}__{x[2]}'  # !!! dunder!!!
    if (x[2] != '')
    else f'{x[0]}__'
    for x in dfs_episode5.columns.to_flat_index()
]
# df_episode_new.index = [
#     f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
#     if x[1] != ''
#     else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
#     for x in dfs_episode5.index.to_flat_index()
# ]
# df_episode_new.dtypes
dfs_episode5
dfs_episode5.index


In [ ]:

dfs_episode6 = dfs_episode4.copy()
dfs_episode6.index = dfs_episode6.index.set_levels(
    [episodestart - pd.Timedelta(1, 'D')-pd.Timedelta(4, 'H')], level='episodestart'
)
dfs_episode6.index = dfs_episode6.index.set_levels(
    [[trucks_by_id['MP73'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
# dfs_episode6 = dfs_episode5.swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=0)
dfs_episode6 = dfs_episode6.reset_index(level=['vehicle', 'driver', 'episodestart'])

dfs_episode6['vehicle'] = dfs_episode6['vehicle'].astype('category')
dfs_episode6['driver'] = dfs_episode6['driver'].astype('category')
# dfs_episode6['episodestart'] = dfs_episode6['episodestart'].astype('category')
dfs_episode6.dtypes
# encoding columns index
dfs_episode6.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '' and x[2] != '')
    else f'{x[0]}_{x[1]}_'
    if (x[1] != '' and x[2] == '')
    else f'{x[0]}__{x[2]}'  # !!! dunder!!!
    if (x[2] != '')
    else f'{x[0]}__'
    for x in dfs_episode6.columns.to_flat_index()
]
# dfs_episode6.index = [
#     f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
#     if x[1] != ''
#     else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
#     for x in dfs_episode6.index.to_flat_index()
# ]
# df_episode_new.dtypes
dfs_episode6


#### encoding single dataframe of multiindex into dask single index dataframe

In [ ]:
ddf_read = dd.read_parquet(
    'eos_new_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
# df_all = ddf_read.compute()
# df_all
# idx = pd.IndexSlice
# col_epistart = ddf_read.loc[:, 'episodestart__'].astype('datetime64[ns]').compute()
# col_epistart
#
# ddf_read.loc[:, 'episodestart__'] = ddf_read.loc[:, 'episodestart__'].astype(
#     'datetime64[ns]'
# )
ddf_read['episodestart__'] = ddf_read['episodestart__'].astype('datetime64[ns]')
ddf_read.compute()


In [ ]:
# ddf_episodes_new = dd.concat([dfs_episode5, dfs_episode6])
# ddf_combine = dd.concat([ddf_read, ddf_episodes_new])
ddf_read = dd.concat([ddf_read, dfs_episode5, dfs_episode6])
ddf_read.compute()
ddf_read.dtypes

In [ ]:
# ddf_list = []
# for n in range(ddf_read.npartitions):
#     ddf_list.append(ddf_read.partitions[n])
#     # ddf_list[n].compute()
# len(ddf_list)
# type(ddf_list[0])

In [ ]:
# import dask
# df_new_list = [dfs_episode5, dfs_episode6]
# df_attach = []
# for df in df_new_list:
#     bHit = False
#     for i, ddf in enumerate(ddf_read.to_delayed()):
#         # if dfs_episode_new['vehicle']
#         print(f"{i}")
#         # df = ddf.compute()
#         # ddf.columns
#         # ddf.dtypes
#         # type(ddf)
#         vehicle = ddf['vehicle__'].compute()[0]
#         driver = ddf['driver__'].compute()[0]
#         print(f"new {df['vehicle__'][0]} in {df['driver__'][0]} vs. pq {vehicle} {driver}")
#
#         # df_episode_new = df_episode_new.append(ddf)
#         if df['vehicle__'][0] == vehicle and df['driver__'][0] == driver:
#             bHit = True
#             print(
#                 f'hit {i}{"st" if i==1 else "nd" if i==2 else "rd" if i==3 else "th"} partition'
#             )
#             # ddf_list[i] = dd.concat([ddf, df], axis=0)
#             # df_attach.append(df)
#             # ddf = dd.from_delayed(dask.delayed(pd.concat)([ddf, df], axis=0),meta=ddf_read._meta)  # !!! not working, cannot change the specific partition!
#
#
#     if bHit == False:
#         print('no hit')
#         df_attach.append(df)
#         # append ddf to dfs_episode_new


In [ ]:

# ddf_read = dd.concat([ddf_read, *df_attach, dfs_episode6])


In [ ]:
try:
     with ProgressBar():
         ddf_read.to_parquet(
             'eos_complete_episodes_added',
             engine='pyarrow',
             compression='snappy',
             partition_on=['vehicle__', 'driver__', 'episodestart__'],
             write_metadata_file=True,
             custom_metadata=custom_metadata,
             append=True,
         )

except Exception as e:
    print(e)
 # append ddf to dfs_episode_new

In [ ]:
ddf_read.compute()

In [ ]:
# res = [ddf_read.columns == ddf_read_new.columns]
# res

In [ ]:
# ddf_list.append(df_attach[0])
# ddf_list[0].dtypes
# ddf_all = dd.concat(ddf_list, axis=0, join='outer')

In [ ]:

# ddf_all.compute()os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[8, 800, 8],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_read.to_parquet(
            'eos_combine_episodes_added',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle__', 'driver__', 'episodestart__'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)


In [ ]:
ddf = dd.read_parquet(
    'eos_combine_episodes_added',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
df = ddf.compute()
df

In [ ]:
# ddf_all.compute()
ddf = dd.read_parquet(
    'eos_combine_episodes_added',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('eos_new_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

df = ddf.compute()

multi_tpl = [tuple(col.split('_')) for col in ddf.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i1 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1
]  # convert index of level 2 type to int and '' if NA
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA

multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['qtuple', 'rows', 'idx']
df.columns = multi_col

# multi_idx = [tuple(idx.split('_')) for idx in ddf.index]
# multi_idx = pd.MultiIndex.from_tuples(multi_idx)
# i0 = multi_idx.get_level_values(
#     0
# )  # must be null string instead of the default pd.NA or np.nan
# i0 = [
#     ''
#     if str(idx) in (str(pd.NA), 'nan', '')
#     else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
#     for idx in i0
# ]  # convert index of level 2 type to int and '' if NA
# i1 = multi_idx.get_level_values(
#     1
# )  # must be null string instead of the default pd.NA or np.nan
# i1 = [
#     ''
#     if str(idx) in (str(pd.NA), 'nan', '')
#     else datetime.strptime(idx, '%Y-%m-%dT%H:%M:%S.%f')
#     for idx in i1
# ]  # convert index of level 2 type to int and '' if NA
# multi_idx = pd.MultiIndex.from_arrays([i0, i1], names=('episodestart', 'timestamp'))
# df.index = multi_idx


df = df.set_index(['vehicle', 'driver', 'episodestart', df.index])
df
df.columns
df.index
# len(multi_idx)


## Sample Quadratuple from parquet files

In [ ]:

ddf = dd.read_parquet(
    'eos_new_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
df = ddf.compute()
df = df.sort_index()
df

In [ ]:
ddf_list = []
for n in range(ddf.npartitions):
    ddf_list.append(ddf.partitions[n])
len(ddf_list)
type(ddf_list[0])


In [ ]:
ddf.compute()

In [ ]:
ddf[['vehicle__','driver__','episodestart__']].compute()

## slicing from dask dataframe

In [ ]:
timestamp_slice = pd.date_range(ts, periods=3, freq='H') + pd.Timedelta(10, 'min')
timestamp_slice
ep_start = ts - pd.Timedelta(6, 'D')
ep_start
# time_slice
episode_slice = pd.date_range(ep_start, periods=6, freq='D')
episode_slice

In [ ]:
ddf['episodestart__'] = ddf['episodestart__'].astype('datetime64[ns]')

In [ ]:
dfs_selected_episodes = ddf.loc[
    (ddf['driver__'] == drivers_by_id['wang-cheng'].pid)
    & (ddf['vehicle__'] == trucks_by_id['VB7'].vid)
]
dfs_selected_episodes.compute()

In [ ]:
dfs_selected_episodes = ddf.loc[
    (ddf['driver__'] == drivers_by_id['zheng-longfei'].pid)
    & (ddf['episodestart__'] < episode_slice[5])
    & (ddf['episodestart__'] > episode_slice[0])

]
episode_slice[5]
dfs_selected_episodes.compute()
len(dfs_selected_episodes)

In [ ]:
dfs_selected_episodes = ddf.loc[
    (ddf['driver__'] == drivers_by_id['zheng-longfei'].pid)
    & (ddf['episodestart__'] < episode_slice[5])
    & (ddf['episodestart__'] > episode_slice[0])
].loc[timestamp_slice[0]:timestamp_slice[2]]
dfs_selected_episodes.compute()
len(dfs_selected_episodes)

In [170]:
idx = pd.IndexSlice
dfs_selected_episodes = ddf.loc[
    (ddf['driver__'] == drivers_by_id['wang-cheng'].pid)
    & (ddf['vehicle__'] == trucks_by_id['VB7'].vid)
]
dfs_selected_episodes.compute()

,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,action_r1_3,action_r1_4,...,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle__,driver__,episodestart__
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-07-04 15:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-06-28 14:48:53.131895
2023-07-04 16:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-06-28 14:48:53.131895
2023-07-04 17:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-06-28 14:48:53.131895
2023-07-04 18:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-06-28 14:48:53.131895
2023-07-04 19:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-06-28 14:48:53.131895
2023-07-04 15:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-07-04 14:48:52.594851
2023-07-04 16:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-07-04 14:48:52.594851
2023-07-04 17:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-07-04 14:48:52.594851
2023-07-04 18:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-07-04 14:48:52.594851


In [178]:
batch = dfs_selected_episodes.sample(frac=0.4)
batch.compute()

,action_r0_0,action_r0_1,action_r0_2,action_r0_3,action_r0_4,action_r1_0,action_r1_1,action_r1_2,action_r1_3,action_r1_4,...,state_timestep_1,state_timestep_2,state_timestep_3,state_velocity_0,state_velocity_1,state_velocity_2,state_velocity_3,vehicle__,driver__,episodestart__
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-07-04 19:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-06-28 14:48:53.131895
2023-07-04 15:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-06-28 14:48:53.131895
2023-07-04 18:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-07-04 14:48:52.594851
2023-07-04 16:48:52.594851,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,...,2023-07-04 14:48:52.614851,2023-07-04 14:48:52.634851,2023-07-04 14:48:52.654851,0.0,1.0,2.0,3.0,VB7,wang-cheng,2023-07-04 14:48:52.594851


In [173]:
def decode_dataframe_from_parquet(df: pd.DataFrame):
    """
    decode the dataframe from parquet with flat column indices to MultiIndexed DataFrame
    """

    multi_tpl = [tuple(col.split('_')) for col in df.columns]
    multi_col = pd.MultiIndex.from_tuples(multi_tpl)
    i1 = multi_col.get_level_values(0)
    i1 = [
        '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1
    ]  # convert index of level 2 type to int and '' if NA
    i2 = multi_col.get_level_values(
        1
    )  # must be null string instead of the default pd.NA or np.nan
    i2 = [
        '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
    ]  # convert index of level 2 type to int and '' if NA
    i3 = multi_col.get_level_values(
        2
    )  # must be null string instead of the default pd.NA or np.nan
    i3 = [
        '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
    ]  # convert index of level 2 type to int and '' if NA

    multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
    multi_col.names = ['qtuple', 'rows', 'idx']
    df.columns = multi_col

    df = df.set_index(['vehicle', 'driver', 'episodestart', df.index])

    return df


In [180]:
df = batch.compute()
batch = decode_dataframe_from_parquet(df)

tuple                                                                    action  \
rows                                                                         r0   
idx                                                                           0   
vehicle driver     episodestart               timestamp                           
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851   12.0   
                                              2023-07-04 15:48:52.594851   12.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851   12.0   
                                              2023-07-04 16:48:52.594851   12.0   

tuple                                                                           \
rows                                                                             
idx                                                                          1   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  13.0   
                                              2023-07-04 15:48:52.594851  13.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  13.0   
                                              2023-07-04 16:48:52.594851  13.0   

tuple                                                                           \
rows                                                                             
idx                                                                          2   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  14.0   
                                              2023-07-04 15:48:52.594851  14.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  14.0   
                                              2023-07-04 16:48:52.594851  14.0   

tuple                                                                           \
rows                                                                             
idx                                                                          3   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  15.0   
                                              2023-07-04 15:48:52.594851  15.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  15.0   
                                              2023-07-04 16:48:52.594851  15.0   

tuple                                                                           \
rows                                                                             
idx                                                                          4   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  16.0   
                                              2023-07-04 15:48:52.594851  16.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  16.0   
                                              2023-07-04 16:48:52.594851  16.0   

tuple                                                                           \
rows                                                                        r1   
idx                                                                          0   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  17.0   
                                              2023-07-04 15:48:52.594851  17.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  17.0   
                                              2023-07-04 16:48:52.594851  17.0   

tuple                                             

In [181]:
batch
batch.index
batch.columns

tuple                                                                    action  \
rows                                                                         r0   
idx                                                                           0   
vehicle driver     episodestart               timestamp                           
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851   12.0   
                                              2023-07-04 15:48:52.594851   12.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851   12.0   
                                              2023-07-04 16:48:52.594851   12.0   

tuple                                                                           \
rows                                                                             
idx                                                                          1   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  13.0   
                                              2023-07-04 15:48:52.594851  13.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  13.0   
                                              2023-07-04 16:48:52.594851  13.0   

tuple                                                                           \
rows                                                                             
idx                                                                          2   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  14.0   
                                              2023-07-04 15:48:52.594851  14.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  14.0   
                                              2023-07-04 16:48:52.594851  14.0   

tuple                                                                           \
rows                                                                             
idx                                                                          3   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  15.0   
                                              2023-07-04 15:48:52.594851  15.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  15.0   
                                              2023-07-04 16:48:52.594851  15.0   

tuple                                                                           \
rows                                                                             
idx                                                                          4   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  16.0   
                                              2023-07-04 15:48:52.594851  16.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  16.0   
                                              2023-07-04 16:48:52.594851  16.0   

tuple                                                                           \
rows                                                                        r1   
idx                                                                          0   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  17.0   
                                              2023-07-04 15:48:52.594851  17.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  17.0   
                                              2023-07-04 16:48:52.594851  17.0   

tuple                                             

MultiIndex([('VB7', 'wang-cheng', '2023-06-28 14:48:53.131895', ...),
            ('VB7', 'wang-cheng', '2023-06-28 14:48:53.131895', ...),
            ('VB7', 'wang-cheng', '2023-07-04 14:48:52.594851', ...),
            ('VB7', 'wang-cheng', '2023-07-04 14:48:52.594851', ...)],
           names=['vehicle', 'driver', 'episodestart', 'timestamp'])

MultiIndex([('action',       'r0', 0),
            ('action',       'r0', 1),
            ('action',       'r0', 2),
            ('action',       'r0', 3),
            ('action',       'r0', 4),
            ('action',       'r1', 0),
            ('action',       'r1', 1),
            ('action',       'r1', 2),
            ('action',       'r1', 3),
            ('action',       'r1', 4),
            ('action',       'r2', 0),
            ('action',       'r2', 1),
            ('action',       'r2', 2),
            ('action',       'r2', 3),
            ('action',       'r2', 4),
            ('action',    'speed', 0),
            ('action',    'speed', 1),
            ('action',    'speed', 2),
            ('action', 'throttle', 0),
            ('action', 'throttle', 1),
            ('action', 'throttle', 2),
            ('action', 'throttle', 3),
            ('action', 'throttle', 4),
            ('action', 'timestep', 0),
            ('action', 'timestep', 1),
            ('action', 't

In [190]:
batch['state']

rows                                                                     brake  \
idx                                                                          0   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851   8.0   
                                              2023-07-04 15:48:52.594851   8.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851   8.0   
                                              2023-07-04 16:48:52.594851   8.0   

rows                                                                           \
idx                                                                         1   
vehicle driver     episodestart               timestamp                         
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  9.0   
                                              2023-07-04 15:48:52.594851  9.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  9.0   
                                              2023-07-04 16:48:52.594851  9.0   

rows                                                                            \
idx                                                                          2   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  10.0   
                                              2023-07-04 15:48:52.594851  10.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  10.0   
                                              2023-07-04 16:48:52.594851  10.0   

rows                                                                            \
idx                                                                          3   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  11.0   
                                              2023-07-04 15:48:52.594851  11.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  11.0   
                                              2023-07-04 16:48:52.594851  11.0   

rows                                                                     thrust  \
idx                                                                           0   
vehicle driver     episodestart               timestamp                           
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851    4.0   
                                              2023-07-04 15:48:52.594851    4.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851    4.0   
                                              2023-07-04 16:48:52.594851    4.0   

rows                                                                           \
idx                                                                         1   
vehicle driver     episodestart               timestamp                         
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  5.0   
                                              2023-07-04 15:48:52.594851  5.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  5.0   
                                              2023-07-04 16:48:52.594851  5.0   

rows                                                                           \
idx                                                                         2   
vehicle driver     episodestart               timestamp                         
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  6.0   
                                              2023-07-04 15:48:52.594851  6.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  6.0   
                                              2023-07-04 16:48:52.59485

In [204]:
batch.loc[:, idx['state', ['velocity', 'thrust', 'brake']]].values.astype(np.float32)

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.]],
      dtype=float32)

In [196]:
batch['action']

rows                                                                        r0  \
idx                                                                          0   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  12.0   
                                              2023-07-04 15:48:52.594851  12.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  12.0   
                                              2023-07-04 16:48:52.594851  12.0   

rows                                                                            \
idx                                                                          1   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  13.0   
                                              2023-07-04 15:48:52.594851  13.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  13.0   
                                              2023-07-04 16:48:52.594851  13.0   

rows                                                                            \
idx                                                                          2   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  14.0   
                                              2023-07-04 15:48:52.594851  14.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  14.0   
                                              2023-07-04 16:48:52.594851  14.0   

rows                                                                            \
idx                                                                          3   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  15.0   
                                              2023-07-04 15:48:52.594851  15.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  15.0   
                                              2023-07-04 16:48:52.594851  15.0   

rows                                                                            \
idx                                                                          4   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  16.0   
                                              2023-07-04 15:48:52.594851  16.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  16.0   
                                              2023-07-04 16:48:52.594851  16.0   

rows                                                                        r1  \
idx                                                                          0   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  17.0   
                                              2023-07-04 15:48:52.594851  17.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  17.0   
                                              2023-07-04 16:48:52.594851  17.0   

rows                                                                            \
idx                                                                          1   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-06-28 14:48:53.131895 2023-07-04 19:48:52.594851  18.0   
                                              2023-07-04 15:48:52.594851  18.0   
                   2023-07-04 14:48:52.594851 2023-07-04 18:48:52.594851  18.0   
                                              2023-07-04 1

In [203]:
torque_table_row_names = ['r0', 'r1', 'r2']
batch.loc[:, idx['action', torque_table_row_names]].values.astype(np.float32)

array([[12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.,
        25., 26.],
       [12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.,
        25., 26.],
       [12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.,
        25., 26.],
       [12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.,
        25., 26.]], dtype=float32)

In [202]:
batch.loc[:, idx['nstate', ['velocity', 'thrust', 'brake']]].values.astype(np.float32)

array([[28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
       [28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
       [28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
       [28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.]],
      dtype=float32)

In [201]:
batch.loc[:, idx['reward', 'work']].values.astype(np.float32)

array([[27.],
       [27.],
       [27.],
       [27.]], dtype=float32)

## drop records

In [ ]:
ddf.compute()

In [ ]:
ddf = ddf.drop(dfs_selected_episodes.index)
ddf.compute()

## Sample Quadratuple from parquet files


In [ ]:
target_vehicle = 'VB7'
target_driver = 'wang-cheng'
partition = []
for i, ddf in enumerate(ddf_list):
    # if dfs_episode_new['vehicle']
    print(f"{i}")
    # df = ddf.compute()
    # ddf.columns
    # ddf.dtypes
    vehicle = ddf['vehicle__'].compute()[0]
    driver = ddf['driver__'].compute()[0]
    # df_episode_new = df_episode_new.append(ddf)
    if target_vehicle == vehicle and target_driver == driver:
        bHit = True
        print(
            f'hit {i}{"st" if i==1 else "nd" if i==2 else "rd" if i==3 else "th"} partition'
        )
        # k = i
        partition.append(i)

        # ddf_list[i] = dd.concat([ddf, df], axis=0)
        # append ddf to dfs_episode_new

if bHit == False:
    print('no hit')
    # ddf_list.append(df_episode_new)
    # append ddf to dfs_episode_new

partition
ddf_target = dd.concat([ddf_list[i] for i in partition], axis=0)
# ddf_target
# # ddf_target.head()
df_target = ddf_target.compute()
df_target
# # df_target

# #     c_ep = df_episode_new['ep
# #     dfs_episode_new = dfs_episode_newappend(ddf)
# # print(f"splicing dfs_episode_new {c_ep} with


In [ ]:
tuple_sample = ddf_target.sample(frac=0.2).compute()
tuple_sample['episodestart__']

In [ ]:
ddf_target.sample(frac=0.2).compute()
